In this assignment students will build the random forest model after
normalizing the variable to house pricing from boston data set.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
from sklearn import datasets
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [2]:
boston = datasets.load_boston()

In [4]:
features = pd.DataFrame(boston.data,columns=boston.feature_names)

In [5]:
print("Keys:", boston.keys())
print("Shape:", boston.data.shape) 
print("Columns", boston.feature_names) 

Keys: dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])
Shape: (506, 13)
Columns ['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [6]:
bos = pd.DataFrame(boston.data, columns=boston.feature_names)
bos['PRICE'] = boston.target 
bos.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [7]:
std_price = np.std(boston.target)

In [8]:
X= bos.drop('PRICE', axis=1)
y = bos['PRICE']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 25)
X_scaler = StandardScaler() 
X_train = X_scaler.fit_transform(X_train) 
X_test = X_scaler.transform(X_test)
y_scaler = StandardScaler() 
y_train = y_scaler.fit_transform(y_train[:, None])[:, 0] 
y_test = y_scaler.transform(y_test[:, None])[:, 0]

<ipython-input-9-ed90affa19d4>:6: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  y_train = y_scaler.fit_transform(y_train[:, None])[:, 0]
<ipython-input-9-ed90affa19d4>:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  y_test = y_scaler.transform(y_test[:, None])[:, 0]


Instantiate a random forest regressor since we have to predic on continous variables, and fit the training set

In [10]:
model = RandomForestRegressor() 
model.fit(X_train, y_train) 
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None, oob_score=False, random_state=None, verbose=0, warm_start=False) 
y_pred = model.predict(X_test)
print("Test Accuracy:", format(metrics.r2_score(y_test, y_pred) * 100, '.2f'), '%') 
print("Mean Squared Error:", format(metrics.mean_squared_error(y_test, y_pred), '.5f')) 

Test Accuracy: 86.05 %
Mean Squared Error: 0.11902


Performing gridserach to tune the hyper parameters, then use the best estimator for scoring on the test set.

In [11]:
from sklearn.model_selection import GridSearchCV 
import warnings 
warnings.filterwarnings('ignore')
parameters = {"min_samples_split": [2, 5, 10], "min_samples_leaf": [1, 3, 5], "max_features": ['auto', 'sqrt', 'log2'], "n_estimators": [50, 75, 100] }

grid_search = GridSearchCV(RandomForestRegressor(), param_grid=parameters, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print("Best parameters set found on development set:\n") 
print(grid_search.best_params_)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   25.4s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   49.2s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 1620 out of 1620 | elapsed:  3.0min finished


Best parameters set found on development set:

{'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}


In [12]:
print("Accuracy for test data set:\n") 
y_pred = grid_search.predict(X_test) 
print("Test Accuracy:", format(metrics.r2_score(y_test, y_pred) * 100, '.2f'), '%') 
print("Mean Squared Error:", format(metrics.mean_squared_error(y_test, y_pred), '.5f'))

Accuracy for test data set:

Test Accuracy: 84.15 %
Mean Squared Error: 0.13527
